# Code explanation for the pyy-vs-ly folder

## Requirements

- Needs to have a **log.lammps** file, with `pxx` `pyy` and `pzz` in the arguments of the `thermo_style` command, inside the **pyy-vs-ly** folder to run the code. *You can also change the path in the **main.py** file.*
- Needs **function.py** in the same folder as **main.py** to run.
- Needs `matplotlib` to print graphs. Run the following command in the terminal to install the library: `pip install matplotlib`.
- Needs `linecache` to read the file. Run the following command in the terminal to install the library: `pip install linecache`.

## function.py

This file contains all the functions used by the **main.py** file. Each functions are commented.

In [ ]:
import linecache
import numpy as np 

def split_atom_info_head(info):
    """Atom info string -> Atom info list"""
    res = info.strip('\n')
    atom = res.split('\t')
    return atom

def split_atom_info(info):
    """Atom info string -> Atom info list"""
    res = info.strip('\n')
    atom = res.split(' ')
    return atom


def extract_atoms_info(log_filename):
    """Extracts all infos from the log.lammps file. Returns an array"""
    info = []
    run_thermo = int(split_atom_info_head(linecache.getline(log_filename,211))[2])
    run = int(split_atom_info_head(linecache.getline(log_filename,227))[2])
    nb_values = int(run/run_thermo)

    for i in range(230, 230+nb_values+1):
        temp = split_atom_info(linecache.getline(log_filename,i))
        temp = [i for i in temp if i != '']
        info.append(temp)

    return np.array(info)

def get_pyy_ly(atoms_array):
    pyy = []
    ly = []
    for i in range(len(atoms_array)):
        pyy.append(-float(atoms_array[i][2]))
        ly.append(float(atoms_array[i][9]))
    return pyy, ly

def get_l0(atoms_array):
    """Returns the value of l0"""
    return float(atoms_array[0][9])

def young_module(list_pyy, list_ly, l0):
    """Returns the value of the Young Module"""
    young = []
    eps = []
    delta_eps = []
    delta_pyy = []
    for i in range(len(list_ly)):
        eps_i = (list_ly[i]-l0)/l0
        eps.append(eps_i)

    for j in range(70,len(eps)-1):
        delta_eps.append(eps[j+1] - eps[j])
        delta_pyy.append(list_pyy[j+1] - list_pyy[j])
    
    for k in range(len(delta_eps)):
        young.append(delta_pyy[k]/delta_eps[k])
    return np.mean(young), eps, len(list_pyy)

## main.py

This file contains the code that runs functions in the correct order to print a final graph. This code is self-explanatory thanks to the comments. 

In [ ]:
from function import *
import matplotlib.pyplot as plt 

# Define Variables
# filenames = [r'log.lammps-0,8', r'log.lammps-0,6', r'log.lammps-0,4', r'log.lammps-0,2', r'log.lammps-0,1', r'log.lammps']
filenames = [r'log.lammps_reaxffV4']

# Process
for file in filenames:
    info = extract_atoms_info(file)
    pyy, ly = get_pyy_ly(info)
    if file == filenames[0]:
        young, eps, len = young_module(pyy, ly, get_l0(info))        
        plt.scatter(eps, pyy)
        plt.plot(eps,pyy, label= '0,0 bin')
        plt.xlabel('eps_y')
        print(young)
        print(len)
    else :
        plt.scatter(ly, pyy)
        plt.plot(ly,pyy, label= '0,' + file[13] + ' bin')
        plt.xlabel('ly')

# Plot
plt.ylabel('pyy')
plt.legend(loc='best')
plt.title('pyy vs eps_y')
plt.show()